# 池化
:label:`sec_pooling`

在很多情况下，我们的最终任务是对图像提出一些全局性的问题，例如，*它是否包含一只猫？* 因此，我们最后一层的单元应该对整个输入敏感。通过逐渐聚合信息，生成越来越粗糙的地图，我们实现了最终学习全局表示的目标，同时保留了卷积层在处理中间层的所有优势。在网络中越深，每个隐藏节点敏感的感受野（相对于输入）就越大。降低空间分辨率会加速这一过程，因为卷积核覆盖的有效面积更大。

此外，在检测较低层次的特征时，比如边缘（如:numref:`sec_conv_layer`中讨论的），我们通常希望我们的表示在某种程度上对平移具有不变性。例如，如果我们取一个黑白分明的图像`X`，并将整个图像向右移动一个像素，即`Z[i, j] = X[i, j + 1]`，那么新图像`Z`的输出可能会大不相同。边缘将移动一个像素。实际上，物体几乎不会出现在完全相同的位置。事实上，即使使用三脚架和静止的物体，由于快门移动引起的相机震动也可能使一切移动一个像素左右（高端相机配备了特殊功能来解决这个问题）。

本节介绍*池化层*，它们有两个目的：减轻卷积层对位置的敏感性，并对表示进行空间下采样。

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

## 最大池化和平均池化

与卷积层类似，*池化*操作符
由一个固定形状的窗口组成，该窗口按照其步幅
滑过输入的所有区域，
为每个被固定形状窗口（有时称为*池化窗口*）遍历的位置计算一个输出。
然而，与卷积层中输入和核的互相关计算不同，
池化层不包含参数（没有*核*）。
相反，池化操作符是确定性的，
通常计算池化窗口内元素的最大值或平均值。
这些操作分别称为*最大池化*（简称*max-pooling*）
和*平均池化*。

*平均池化*几乎与CNN一样古老。这个想法类似于
对图像进行下采样。与其只取每隔一个（或三个）
像素的值来生成低分辨率图像，我们可以对相邻像素求平均，
以获得具有更好信噪比的图像，因为我们结合了多个相邻像素的信息。
*最大池化*是在 :citet:`Riesenhuber.Poggio.1999` 中提出的，
在认知神经科学的背景下描述信息如何分层聚合以用于物体识别；
在语音识别中已经有一个早期版本 :cite:`Yamaguchi.Sakamoto.Akabane.ea.1990`。
在几乎所有情况下，最大池化（也称为最大池化）优于平均池化。

在这两种情况下，与互相关运算符一样，
我们可以将池化窗口视为从输入张量的左上角开始，
并从左到右、从上到下滑动。
每当池化窗口到达一个位置时，
它会根据使用的是最大池化还是平均池化，
计算窗口内输入子张量的最大值或平均值。

![使用 $2\times 2$ 池化窗口的最大池化。阴影部分是第一个输出元素以及用于输出计算的输入张量元素：$\max(0, 1, 3, 4)=4$。](../img/pooling.svg)
:label:`fig_pooling`

:numref:`fig_pooling` 中的输出张量高度为2，宽度为2。
四个元素来自每个池化窗口中的最大值：

$$
\max(0, 1, 3, 4)=4,\\
\max(1, 2, 4, 5)=5,\\
\max(3, 4, 6, 7)=7,\\
\max(4, 5, 7, 8)=8.\\
$$

更一般地，我们可以通过在一个所述大小的区域内聚合来定义一个 $p \times q$ 的池化层。
回到边缘检测问题，
我们将卷积层的输出作为 $2\times 2$ 最大池化的输入。
设 `X` 为卷积层的输入，`Y` 为池化层的输出。
无论 `X[i, j]`、`X[i, j + 1]`、
`X[i+1, j]` 和 `X[i+1, j + 1]` 的值是否不同，
池化层总是输出 `Y[i, j] = 1`。
也就是说，使用 $2\times 2$ 最大池化层，
我们仍然可以检测到由卷积层识别的模式
是否在高度或宽度上移动不超过一个元素。

在下面的代码中，我们在 `pool2d` 函数中实现了池化层的前向传播。
这个函数类似于 :numref:`sec_conv_layer` 中的 `corr2d` 函数。
但是，不需要核，而是计算输入中每个区域的最大值或平均值作为输出。

In [2]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

我们可以构建输入张量`X`在:numref:`fig_pooling`中来[**验证二维最大池化层的输出**]。

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

另外，我们还可以试验（**平均池化层**）。

In [4]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

## [**填充和步幅**]

与卷积层一样，池化层
会改变输出形状。
同样地，我们可以通过填充输入和调整步幅来
实现所需的输出形状。
我们可以通过深度学习框架中内置的二维最大池化层
来展示在池化层中使用填充和步幅的效果。
我们首先构建一个四维形状的输入张量`X`，
其中示例数量（批量大小）和通道数都为1。

In [5]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

由于池化是从一个区域汇总信息，（**深度学习框架默认匹配池化窗口大小和步幅。**）例如，如果我们使用形状为 `(3, 3)` 的池化窗口，默认情况下我们会得到形状为 `(3, 3)` 的步幅。

In [6]:
pool2d = nn.MaxPool2d(3)
# Pooling has no model parameters, hence it needs no initialization
pool2d(X)

tensor([[[[10.]]]])

不用说，[**步幅和填充可以手动指定**]以在需要时覆盖框架默认值。

In [7]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

当然，我们可以指定一个任意高度和宽度的矩形池化窗口，如下例所示。

In [8]:
pool2d = nn.MaxPool2d((2, 3), stride=(2, 3), padding=(0, 1))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

## 多个通道

在处理多通道输入数据时，
[**池化层分别对每个输入通道进行池化**]，
而不是像卷积层那样在通道上对输入求和。
这意味着池化层的输出通道数与输入通道数相同。
下面，我们将张量 `X` 和 `X + 1`
在通道维度上连接起来以构建一个具有两个通道的输入。

In [9]:
X = torch.cat((X, X + 1), 1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

正如我们所见，池化后输出通道的数量仍然是两个。

In [10]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

## 摘要

池化是一个极其简单的操作。它正如其名称所示，是在一个值的窗口上聚合结果。所有的卷积语义，如步幅和填充，都以与之前相同的方式应用。请注意，池化对通道是无感的，即它不会改变通道的数量，并且它是对每个通道分别应用的。最后，在两种流行的池化选择中，最大池化比平均池化更可取，因为它赋予输出某种程度的不变性。一个流行的选择是选取 $2 \times 2$ 的池化窗口大小来将输出的空间分辨率减少四分之一。

需要注意的是，除了池化之外还有许多其他方法可以降低分辨率。例如，在随机池化 :cite:`Zeiler.Fergus.2013` 和分数最大池化 :cite:`Graham.2014` 中，聚合与随机化结合在一起。这在某些情况下可以稍微提高准确性。最后，正如我们将在后面看到的注意力机制中所见，有更精细的方法来聚合输出，例如通过使用查询和表示向量之间的对齐。

## 练习

1. 通过卷积实现平均池化。
1. 证明仅通过卷积无法实现最大池化。
1. 可以使用ReLU操作来完成最大池化，即 $\textrm{ReLU}(x) = \max(0, x)$。
    1. 仅使用ReLU操作表达 $\max (a, b)$。
    1. 使用这种方法通过卷积和ReLU层实现最大池化。
    1. 对于 $2 \times 2$ 卷积需要多少个通道和层数？对于 $3 \times 3$ 卷积呢？
1. 池化层的计算成本是多少？假设输入到池化层的尺寸为 $c\times h\times w$，池化窗口的形状为 $p_\textrm{h}\times p_\textrm{w}$，填充为 $(p_\textrm{h}, p_\textrm{w})$，步幅为 $(s_\textrm{h}, s_\textrm{w})$。
1. 为什么你预期最大池化和平均池化工作方式不同？
1. 我们需要一个单独的最小池化层吗？你能用另一个操作替换它吗？
1. 我们可以使用softmax操作来进行池化。为什么它可能不那么受欢迎？

[讨论](https://discuss.d2l.ai/t/72)